## This notebook shows how to use Orchestrator APIs for user experiments

In [ ]:
import os, traceback
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [ ]:
slice_name="Slice-comps-only"

In [ ]:
fabric_rc_location=os.environ['HOME']+"/work/fabric_config/fabric_rc"
if os.path.exists(fabric_rc_location):
    with open(fabric_rc_location, 'r') as f:
        for line in f:
            if line.startswith('export'):
                os.environ[line.split('=')[0].split('export')[1].strip()] = line.split('=')[1].strip()

In [ ]:
ssh_key_file_priv=os.environ["FABRIC_SLICE_PRIVATE_KEY_FILE"]
ssh_key_file_pub=os.environ["FABRIC_SLICE_PUBLIC_KEY_FILE"]

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

In [ ]:
print(os.environ["FABRIC_PROJECT_ID"])

## Create Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [ ]:
slice_manager = SliceManager()

### Orchestrator API example to query for available resources

In [ ]:
status, advertised_topology = slice_manager.resources(force_refresh=True)

print(f"Status: {status}")
if status == Status.OK:
    print(f"Toplogy: {advertised_topology}")
else:
    print(f"Error: {advertised_topology}")

In [ ]:
if status == Status.OK:
    advertised_topology.draw()

## Create Slice

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType, Labels, Flags


# Create topology
t = ExperimentTopology()
# Set capacities
cap = Capacities(core=60, ram=6, disk=10)

boot_script = f"sudo yum install -y yum-utils;" \
         f"sudo yum-config-manager --add-repo https://download.docker.com/linux/centos/docker-ce.repo;" \
         f"sudo yum install -y docker-ce docker-ce-cli containerd.io docker-compose-plugin;" \
         f"sudo systemctl start docker;" \
         f"sudo systemctl enable docker;" \
         f"sudo curl -SL https://github.com/docker/compose/releases/download/v2.5.0/docker-compose-linux-x86_64 -o /usr/local/bin/docker-compose;" \
         f"sudo chmod +x /usr/local/bin/docker-compose;" \


sites = ['RENC']
labels = Labels(instance_parent="lbnl-w3.fabric-testbed.net")
name="node"
i=0
for s in sites:
    # Add node
    nm = f"{name}-{i}"
    
    n = t.add_node(name=f"{name}-{i}", site=s)
    

    # Set properties
    n.set_properties(capacities=cap, image_type='qcow2', image_ref='default_rocky_8')
    #n.set_properties(capacities=cap, image_type='qcow2', image_ref='default_rocky_8', boot_script=boot_script)
    
    n.add_storage(name='vol2', labels=Labels(local_name='vol1'), flags=Flags(auto_mount=True))
    n.add_component(model_type=ComponentModelType.SmartNIC_ConnectX_6, name=f"{name}-{i}-nic1")
    
    i += 1

# Generate Slice Graph
slice_graph = t.serialize()
slices = []

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name=f'{slice_name}-{i}', slice_graph=slice_graph, ssh_key=ssh_key_pub)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Reservations created {}".format(reservations))
    slice_id = reservations[0].slice_id
    slices.append(slice_id)
else:
    print(f"ERROR: {reservations}")
print(slices)

## Query Slices

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology
status, slices = slice_manager.slices(slice_id=slice_id)
print(slices)
slice_object = slices[0]
topo = ExperimentTopology()
topo.load(graph_string=slice_object.model)
topo.draw()

## Query Slivers

In [ ]:
status, slivers = slice_manager.slivers(slice_object=slice_object)

print("Response Status {}".format(status))
if status == Status.OK:
    for s in slivers:
        print("Name {}".format(s.fim_sliver.get_name()))
        print("Name {}".format(s.fim_sliver.get_management_ip()))
        print(f"State {s.state}")
        print("State {}".format(s.fim_sliver.get_reservation_info()))
        print("Site {}".format(s.fim_sliver.get_site()))
        print("Type {}".format(s.fim_sliver.get_type()))
        print("Labels {}".format(s.fim_sliver.get_labels()))
        print("LabelAllocations {}".format(s.fim_sliver.get_label_allocations()))
        print("Capacities {}".format(s.fim_sliver.get_capacities()))
        print("CapacityHints {}".format(s.fim_sliver.get_capacity_hints()))
        print("CapacityAllocations {}".format(s.fim_sliver.get_capacity_allocations()))
        print("=====================")
        
        if s.sliver_type == "NodeSliver":
            for d in s.fim_sliver.attached_components_info.devices.values():
                print("-------------------------")
                print("COMPONENT        {}".format(d))
                print("-------------------------")
        else:
            for d in s.fim_sliver.interface_info.interfaces.values():
                print("-------------------------")
                print("INTERFACE        {}".format(d))
                print("-------------------------")
                
        print("Notices       {}".format(s.notice))
        print("=====================")
        
else:
    print(f"Failure: {slivers}")

## Delete Slice

In [ ]:
status, result = slice_manager.delete(slice_object=slice_object)

print("Response Status {}".format(status))
print("Response received {}".format(result))